# Borrower reliability assessment

Customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. 

Input data from the bank - statistics on the solvency of customers.

The result of the research will be taken into account when building a **credit scoring** model - a special system that assesses the ability of a potential borrower to repay a loan to a bank.

## Step 1. Data loading and initial analysis

In [4]:
import pandas as pd
data = pd.read_csv(r'C:\Users\chapp\Downloads\data.csv')
data.info()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   с

**Conclusion**

The first thing that catches my eye is the lack of data on monthly income and total work experience in days for more than 2,000 clients.
Some values in the days_employed column are represented by negative numbers, which is not possible. Possible technical error in data collection.
The decision to take into account the length of service in days as a real number also raises questions; the use of integer values seems more rational.
The education column contains values ​​that are capitalized and written entirely in uppercase. It is proposed to format them under the standard view in lowercase. You also need to set the gender column to lower case.

## Шаг 2. Data preprocessing

### Pass processing

In [15]:
data.groupby(['income_type', 'dob_years'])['total_income'].transform('median').value_counts().head(40)

151797.844394    383
145093.114066    378
147847.570664    377
140033.566488    377
141573.270579    375
144254.864743    368
153773.968274    361
142046.360978    359
140068.472941    347
141751.985878    339
153747.902987    338
144728.615470    338
147848.302877    334
144072.660536    330
138532.493785    329
144178.630128    324
145121.091480    321
143761.158553    319
143033.121693    315
140948.448154    314
141497.363438    309
144681.423262    290
142251.512638    289
124094.740661    275
132937.539324    275
115791.112491    268
128882.579913    259
134420.239376    249
128762.733396    246
113668.149752    241
112383.229323    236
143161.907955    231
138874.221779    222
133134.658382    218
116716.982338    210
121653.820122    207
127962.834610    203
136510.867838    203
139547.180595    202
175635.434186    192
Name: total_income, dtype: int64

In [5]:

data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'] * -1  #remove negative values of the seniority column in days
data['days_employed'] = data['days_employed'].fillna(data.groupby(['income_type', 'dob_years'])['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type', 'dob_years'])['total_income'].transform('median'))

#data[data.isnull().any(1)]
#after filling in the missing values, based on the age and type of earnings, there are 4 lines that were not found
#median values. Probably no matching groups found. For example, for a 24-year-old pensioner.
#fill these four lines with median values for the entire frame.

days_emp_med = data['days_employed'].median()                                      
#print (days_emp_med)                                                              
data['days_employed'] = data['days_employed'].fillna(days_emp_med)                 

income_med = data['total_income'].median()                                         
#print (income_med)                                                                
data['total_income'] = data['total_income'].fillna(income_med)                     



data['education'] = data['education'].str.lower() 
#print(data.head(10))

data.loc[data['children'] < 0, 'children'] = data['children'] * -1                 #remove the negative values of the number of children column
#print (data.groupby('children')['debt'].count())
#print (data.loc[data['children'] == 20])
children_med = data.loc[data['children'] != 20, 'children'].median()               
#print (children_med)                                                              #median = 0
data.loc[data['children'] == 20, 'children'] = children_med
               
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Conclusion**

The missing information on the length of service and income of about 2174 borrowers is filled in with the median value for the column. The use of the average value is not possible, since there are large outliers of indicators. To calculate the median value of the days_employed column, work was also done to remove indicators with a negative value and replace them with the same value with a + sign. There are no more missing values.

### Data types changing

In [3]:
data['days_employed'] = data['days_employed'].astype('int')    #замена типа данных float64 столбца days_employed на int64
data.info()
data['children'] = data['children'].astype('int')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null float64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Conclusion**

The float values of seniority in days were replaced with integer ones.

### Duplicate processing

In [4]:
#data.duplicated().sum()
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

0


**Conclusion**

Found 54 duplicate lines. At the moment, after clearing them, they are 0. Also, the strings from the education column are unified to lower case.

### Lemmatization

In [5]:
from pymystem3 import Mystem
m = Mystem()

def lemmas_purpose(row):
    lem_purpose = ' '.join(m.lemmatize(row['purpose']))
 
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    elif 'недвижимость' in lem_purpose or 'жил' in lem_purpose or 'ремонт' in lem_purpose:
        return'недвижимость'
    elif 'образование' in lem_purpose:
        return 'образование'
    else:
        return 'свадьба'
 
data['purpose_category'] = data.apply(lemmas_purpose, axis=1)
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


**Conclusion**

Data on the purpose of the loan are lemmatized and divided into 4 groups. Information about this is reflected in the new column of the table.

### Data categorization

In [6]:
print(data.total_income.describe())

def income_category(row):
    if row['total_income'] <= 1.071352e+05:
        return 'Наименьший доход'
    elif row['total_income'] >= 1.071352e+05 and row['total_income'] <= 1.442549e+05:
        return'Доход ниже среднего'
    elif row['total_income'] >= 1.442549e+05 and row['total_income'] <= 1.961555e+05:
        return'Доход выше среднего'
    else:
        return 'Сверхбольшой доход'
 
data['income_category'] = data.apply(income_category, axis=1)
data.head(10)

count    2.145400e+04
mean     1.653435e+05
std      9.822851e+04
min      2.066726e+04
25%      1.071352e+05
50%      1.442549e+05
75%      1.961555e+05
max      2.265604e+06
Name: total_income, dtype: float64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,Сверхбольшой доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,Доход ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,Доход выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,Сверхбольшой доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,Доход выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость,Сверхбольшой доход
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость,Сверхбольшой доход
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,Доход ниже среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,Наименьший доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,Доход выше среднего


**Conclusion**

Client salaries are broken down into 5 categories. Up to 50,000 rubles, from 50,000 to 100,000, from 100,000 to 200,000, from 200,000 to 500,000, from 500,000 and above. Category information is placed in a new column - income_category.

## Step 3. Answers on questions

- Is there a relationship between having children and repaying a loan on time?

In [7]:
a = data.groupby('children')['debt'].count()
b = data.groupby('children')['debt'].sum()
print(b/a*100)



children
0    7.559822
1    9.165808
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


**Conclusion**

With an increase in the number of children, the likelihood of not repaying the loan on time increases. Dependency exists. (The number of families with 5 children in the sample is too small to conclude that 100% of families with 5 children repay loans on time)

- Is there a relationship between marital status and loan repayment on time?

In [8]:
a = data.groupby('family_status')['debt'].count()
b = data.groupby('family_status')['debt'].sum()
print(b/a*100)


family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64


**Conclusion**

The results do not show a clear pattern. The indicators for "married / married" and "divorced", "widower / widow" are approximately equal. For "Not married / not married" and "civil marriage" are approximately equal. However, it can be assumed that people somehow connected with the institution of official marriage are slightly more likely to repay the loan on time. Perhaps they have more reverence for formal treaties. There is a dependency.

- Is there a relationship between the level of income and repayment of the loan on time?

In [9]:
a = data.groupby('income_category')['debt'].count()
b = data.groupby('income_category')['debt'].sum()
print(b/a*100)

income_category
Доход выше среднего    8.565031
Доход ниже среднего    8.757220
Наименьший доход       7.997763
Сверхбольшой доход     7.140194
Name: debt, dtype: float64


**Conclusion**

Approximately the same indicators for categories earning the least and most.
People with an average income are less likely to repay a loan on time. There is a relationship, but it is not linear.

- How do different purposes of the loan affect its repayment on time?

In [10]:
a = data.groupby('purpose_category')['debt'].count()
b = data.groupby('purpose_category')['debt'].sum()
print(b/a*100)

purpose_category
автомобиль      9.359034
недвижимость    7.233373
образование     9.220035
свадьба         8.003442
Name: debt, dtype: float64


**Conclusion**

Approximately the same indicators for borrowers for the purpose of purchasing a car or education. Slightly less inclined to repay the loan on time are borrowers planning events. Real estate buyers are the most accurate payers. There is a dependency.

## Step 4. Final conclusion

The findings are surprising. The most solvent borrower is a married (and also married or married) childless person with a minimum or super high income who purchases real estate. They are conscientious and accurate payers. But people with many children, unmarried people who are interested in education or cars are hardly the best investment for a lender. I see the connection between these factors in the seriousness of the borrowers' intentions and the tendency to conclude written contracts.